<a href="https://colab.research.google.com/github/shlok-mehndiratta/car_chatbot/blob/main/CAR_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install tools
!apt-get update -qq
!apt-get install -y -qq poppler-utils tesseract-ocr
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

!pip install -q fastapi uvicorn nest_asyncio pdf2image pytesseract python-multipart

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package poppler-utils.
(Reading database ... 121713 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.12_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.12) ...
Setting up poppler-utils (22.02.0-2ubuntu0.12) ...
Processing triggers for man-db (2.10.2-1) ...
Selecting previously unselected package cloudflared.
(Reading database ... 121743 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.11.1) ...
Setting up cloudflared (2025.11.1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
import nest_asyncio
import uvicorn
from fastapi import FastAPI, UploadFile, File
import threading
import time
app = FastAPI()

@app.get("/")
def home():
  return{"message": "Server is running!"}
  nest_asyncio.apply()

In [ ]:
# Simple OCR Helper
from pdf2image import convert_from_path
import pytesseract

def extract_text_from_pdf(pdf_path):
  full= ""
  for p in pages:
    full += pytesseract.image_to_string(p)+"\n"
  return full

In [ ]:
#Upload API
@app.post("/upload")
async def upload_file(file: UploadFile = File(...)):
  path = f"tmp/{file.filename}"
  with open(path, "wb") as f:
    f.write(file.file.read())
  text = extract_text(path) #OCR
  return{"text":text[:2000]} #return first 2000 chars

5th December 2025

In [ ]:
import os
import json
import re
import textwrap
import requests

# Base project folder
BASE_DIR = "/content/CarleaseBot"
CONTRACTS_DIR = os.path.join(BASE_DIR, "contracts")
OUTPUT_DIR = os.path.join(BASE_DIR, "outputs")

os.makedirs(CONTRACTS_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Project folders created:")
print("BASE_DIR     :", BASE_DIR)
print("CONTRACTS_DIR:", CONTRACTS_DIR)
print("OUTPUT_DIR   :", OUTPUT_DIR)

Project folders created:
BASE_DIR     : /content/CarleaseBot
CONTRACTS_DIR: /content/CarleaseBot/contracts
OUTPUT_DIR   : /content/CarleaseBot/outputs


In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 94.0 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import shutil

print("Please upload one or more contract files (PDF preferred).")
uploaded = files.upload()   # choose PDFs from your system

for filename in uploaded.keys():
    src = os.path.join("/content", filename)
    dst = os.path.join(CONTRACTS_DIR, filename)
    shutil.move(src, dst)
    print("Saved:", dst)

print("\nFiles in contracts folder:")
!ls -l "$CONTRACTS_DIR"

Please upload one or more contract files (PDF preferred).


In [ ]:
import pdfplumber

def extract_text_from_pdf(pdf_path: str) -> str:
    """Extracts text from all pages of a PDF file."""
    text_pages = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text() or ""
            text_pages.append(page_text)
    full_text = "\n".join(text_pages)
    return full_text

# Preview first uploaded contract
def preview_first_contract():
    files_in_dir = sorted(os.listdir(CONTRACTS_DIR))
    if not files_in_dir:
        print("No contracts found in:", CONTRACTS_DIR)
        return None, None

    first_file = files_in_dir[0]
    pdf_path = os.path.join(CONTRACTS_DIR, first_file)

    text = extract_text_from_pdf(pdf_path)
    print("Showing first 1000 characters of:", first_file)
    print("-" * 80)
    print(text[:1000])
    print("-" * 80)
    return pdf_path, text

first_pdf_path, first_pdf_text = preview_first_contract()

Showing first 1000 characters of: Car Lease Agreement Template (New).pdf
--------------------------------------------------------------------------------
Car Lease
Agreement
Template
Prepared for Prepared by
[Recipient Name] [Sender Name]
[Company Name] [Company Name]
Car Lease Agreement Template
1 . Introduction
This Car Lease Agreement (the "Agreement") is entered into on this [__ day of _____,
20] by and between [Lessor's Full Name], hereinafter referred to as the "Lessor," and
[Lessee's Full Name], hereinafter referred to as the "Lessee."
This Agreement sets forth the terms and conditions upon which the Lessor agrees to
lease the vehicle described herein to the Lessee, and the Lessee agrees to lease the
said vehicle from the Lessor. The parties hereto mutually agree to abide by all the
terms and conditions stipulated within this Agreement.
2. Description of Vehicle
❖ Vehicle Make: [e.g., Toyota, Ford, etc.]
❖ Vehicle Model: [e.g., Camry, F-150, etc.]
❖ Vehicle Year: [Year of Manufa